# Ray Concepts - Task Parallelism (Part 2)

The previous lesson explored Ray's core concepts and how they work. We learned how to define Ray _tasks_, run them, and retrieve the results. We also started learning about how Ray schedules tasks in a distributed environment.

This lesson completes the discussion of Ray tasks by exploring how task dependencies are handled. We'll finish with a look under the hood at Ray's architecture and runtime behavior.

> **Tip:** Recall that the [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

In [1]:
# If you are running on Google Colab, uncomment and run the following linkes
# to install the necessary dependencies.

# print("Setting up colab environment")
# !pip install -q ray
# !pip install -q bokeh

In [2]:
# Imports and initialize Ray. We're adding NumPy for the examples and the tutorial `util` library:

import ray, time, sys    # New notebook, so new process
import numpy as np       # Used for examples
sys.path.append('..')    # Import our own libraries starting in the project root directory

from util.printing import pnd, pd

In [3]:
ray.init(ignore_reinit_error=True)

2020-04-11 12:25:19,130	INFO resource_spec.py:212 -- Starting Ray with 4.15 GiB memory available for workers and up to 2.1 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-11 12:25:19,483	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:51666',
 'object_store_address': '/tmp/ray/session_2020-04-11_12-25-19_118969_95789/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-11_12-25-19_118969_95789/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-11_12-25-19_118969_95789'}

Let's work with a new remote function. Previously, our `expensive` and `expensive_task` functions returned tuples that included time durations. Obviously the durations were useful for understanding how long the functions took to execute. Now, it will be more convenient to not return "metadata" like this, but just data values that we care about, because we are going to pass them to other functions. 

Hence, we'll define _dependentcy_ relationships between tasks. We'll learn how Ray handles these dependent, asynchronous computations.

So, let's define a task to return a random NumPy array of some size `n`. As before, we'll add a sleep time, one tenth the size of `n`:

In [4]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [5]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [6]:
start = time.time()
id1 = make_array.remote(20)
id2 = make_array.remote(20)
id3 = add_arrays.remote(id1, id2)
print(ray.get(id3))
pd(time.time() - start, prefix="Total time:")

[ 1.58179421  1.04154269 -4.34396502  2.95890516 -0.34284671 -2.16712245
 -0.08134846 -0.36249519  2.48293595 -1.02296294  3.71652342 -0.88654731
  0.04284629 -0.1085318   2.59345773  0.34631241 -0.69452571  2.91697224
  2.84960465 -2.2023927 ]
Total time: duration:  4.023 seconds


Something subtle and "magical" happened here; when we called `add_arrays`, we didn't need to call `ray.get()` first for `id1` and `id2`, since `add_arrays` expects NumPy arrays. Because `add_arrays` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural.

Furthermore, note that the `add_arrays` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_arrays` until the other tasks are finished. Hence, _Ray handles task dependencies automatically for us._ 

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_arrays`.

Recall from the previous lesson that we explored when to call `ray.get()` to avoid forcing tasks to become synchronous when they should be asynchronous. This additional example illustrates two key points:

* _Don't ask for results you don't need._
* _Don't ask for the results you need until you really need them._

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Using ray.wait() with ray.get()

We've seen several examples of the best idiomatic way to use `ray.get()`. Here again is an example from the last lesson:

```python
start = time.time()
ids = [expensive_task.remote(n) for n in range(5)]  # Fire off the asynchronous tasks
for n2, duration in ray.get(ids):                   # Retrieve all the values from the list of futures
    p(n2, duration)
pd(time.time() - start, prefix="Total time:")
```

Let's try it again with our new methods:

In [7]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]
for array in ray.get(added_array_ids):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [ 5.16237778 -0.1610619   0.02745005  1.28288416 -4.5768072  -1.1963492
 -4.14734869 -0.89422203  2.36629564  0.0329063 ]
20: [ 2.37617731  0.21239049 -2.18169847 -1.05908274 -1.01268595  2.37199725
 -2.95181227 -6.09585257 -0.60751517  0.15664815 -0.31481478 -2.15184829
  2.65220041 -2.24464039  1.34548982 -1.37519184  1.66837509  3.07215514
  1.26365101  0.08071517]
30: [-0.37923619  2.39918768 -0.04062384 -3.18274133  2.35223167  0.81657425
  1.24927552 -1.97335395  1.05082757  5.11603432  0.8021905  -2.28633519
  4.06886474 -0.68909667 -0.42209066  0.72501832  1.78771875 -0.20146854
 -2.47293551  3.81555023  0.96088243  1.33613062  1.06570787  5.06590965
 -1.64093351  2.52637303 -0.87703278  0.97992414  1.74535752  2.05481191]
40: [-1.35659297  0.79693151 -1.31405046 -4.64394575  0.68841835 -4.4439996
 -0.35659394  1.92191734 -6.07164238 -3.14075173 -0.05800898  2.31872004
  2.59527647 -2.19605973 -1.75320767 -2.35173182  3.24512472 -0.16844998
  1.95790753  6.05220196 -1

On my machine, I waited 8 seconds and then everything was printed at once.

There are two fundamental problems with the way we've used `ray.get()` so far:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` for waiting ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.wait)) instead, followed by using `ray.get()` to retrieve values for tasks that `ray.wait()` tells us are finished. 

Using `ray.wait()` is also the way to fix the second problem with using `ray.get()` by itself, that we have to wait for all tasks to finish before we get any values back. Some of those tasks might finish quickly, like our contrived examples that sleep for short durations compared to other invocations. 

When you have a list of asynchronous tasks, you want to process the results of them as soon they become available, even while others continue to run. Use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts, for blocking on running tasks. Then use `ray.get()` to retrieve values of completed tasks. Now we'll learn how to use these two together. For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

Here is the previous example rewritten to use `ray.wait()`:

In [8]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=1, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  0.003)
0: []
Returned   1 completed tasks. (elapsed time:  2.012)
10: [-0.38005455 -0.73987206 -0.27129311  1.79471622  1.11799605 -0.05944219
  1.03108081  1.47935947  0.29573855  2.79061424]
Returned   1 completed tasks. (elapsed time:  4.010)
20: [ 1.01443279 -1.77533661 -1.87216986 -1.35347317 -0.29925878 -0.70963494
 -2.00518038 -0.37753958  0.94914918  5.0374975  -1.51269448  3.17682755
  0.99814661  2.53575711  0.48500221  1.77437268 -0.97985738  2.97552256
  0.61717902  2.8247505 ]
Returned   1 completed tasks. (elapsed time:  6.007)
30: [-1.522553   -1.9459189   3.50485027  2.79987737 -1.88213703 -0.41070297
  0.52294885 -0.77777722 -5.02792332 -2.39474844  2.75684085 -1.87893464
 -0.7092303   0.06949743 -0.87534656 -2.16164065 -1.50463455 -3.67631061
  2.37561512 -1.16074439 -1.87046175  0.80861797  1.12991478 -4.20157502
 -1.87181742 -2.95892162 -2.64252699  1.06234088  3.19518274  2.12173205]
Returned   1 completed tasks. (elaps

Now it still takes about 8 seconds to complete, 4 seconds for the longest invocation of `make_array` and 4 seconds for the invocation of `add_arrays`, but since the others complete more quickly, we see their results as soon as they become available, at 0, 2, 4, and 6 second intervals.

> **Warning:** For each call to `ray.wait()` in a loop like this, it's important to remove the ids that have completed. Otherwise, `ray.wait()` will return immediately with the same list containg the first completed item, over and over again; you'll loop forever!! Resetting the list is easy, since the second list returned by `ray.wait()` is the rest of the items that are still running. So, that's what we use.

Now let's try it with `num_returns = 2`:

In [9]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.011)
0: []
10: [-2.43936963 -3.59727277 -1.84808521  0.16980047 -3.00549652  0.01655438
  1.93030098 -2.5789072   0.76485563 -1.05957686]
Returned   2 completed tasks. (elapsed time:  6.014)
20: [ 0.25899647  2.45160359  0.18665095  2.17271455 -2.64571757  1.57135399
 -2.64830768  0.21193865  1.05984056 -1.34923138 -0.03504496 -3.85412121
  0.35523244 -0.63127302 -0.96722465  1.62028204 -0.85957171 -1.31411683
 -0.57116971 -0.05435205]
30: [ 1.93328252 -0.69786817 -1.03359445  0.03693786  0.1100653  -1.00627545
 -0.47008553 -2.25305246 -1.19578224 -0.91578781 -0.27686619  1.71462335
  2.62033208  3.22091971 -0.39390681  1.52723972 -0.76381783  5.85703604
  0.06241465 -2.68903923 -2.80579655  1.69402892 -1.24128659 -1.87926943
 -2.22368385  1.89145385 -3.49478454  1.22666949  2.95226381  3.77575438]
Returned   1 completed tasks. (elapsed time:  8.016)
40: [ 0.11485634 -1.04975051 -0.71700133  0.48439696 -2.43776519  0.11759997
  1.66254359

Now we get two at a time output. Note that we don't actually pass `num_returns=2` every time. If you ask for more items than the length of the input list, you get an error. So, we compute `num_returns`, using `2` except when there's only one task to wait on, in which case we use `1`. So, in fact, the output for `40` was a single task result, because we started with `5` and processed two at a time.

## Exercise 2

The following cell is identical to the last one. Modify it to use a timeout of `2.5` seconds, shorter than our longest tasks. What happens now? Try using other times.

In [10]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.011)
0: []
10: [ 2.43018462  0.18014044 -0.90480674  1.03299245  1.80095756  0.56890552
  0.58775431  2.59994933  2.81104815  2.99965917]
Returned   2 completed tasks. (elapsed time:  6.008)
20: [ 2.50635144  0.86342304 -0.88614723  1.70097258  1.12939558 -1.02829209
  2.29078365  1.22362808  1.68038354  1.96301261  1.53462913  2.47819763
 -2.42531451 -2.38754981 -0.45773884 -1.20321018  0.91021492 -2.09219706
 -0.05620234  1.66995299]
30: [-2.8810452   0.21378184  1.53232417  0.22574913 -2.05205842  5.45095933
  0.91414197 -1.57023602  0.12756271 -5.86076144 -2.15892071 -2.51913047
 -4.6916622   0.43988789  1.46178579 -2.6400027   0.82101144  1.81715691
  2.06758901 -3.55996431 -0.43564825 -0.8417092  -0.51057559 -1.18125218
  1.46064132  2.53190201 -1.46252172 -2.81326894  4.16737664  0.66980033]
Returned   1 completed tasks. (elapsed time:  8.016)
40: [-0.29773063 -0.02141088 -1.11186357 -3.09722678  2.0248422   0.9231757
  0.85521198 

In conclusion:

> **Tips:**
>
> 1. Use `ray.wait()` with a timeout to wait for one or more running tasks. Then use `ray.get()` to retrieve the values for the finished tasks.
> 2. Don't ask for results you don't need.
> 3. Don't ask for the results you need until you really need them.

## Exercise 3

Let's make sure you understand how to use `ray.wait()`. The definitions from Exercise 1 in the previous lesson are repeated in the next cell. Change the definitions to use Ray. In particular, use `ray.wait()` as we used it above. You can just use the default values for `num_returns` and `timeout` if you want. The second cell uses `assert` statements to check your work.

> **Tip:** The solution is in the `solutions` folder.

In [11]:
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
squares = [slow_square(n) for n in range(4)]
duration = time.time() - start

In [12]:
assert squares == [0, 1, 4, 9], f'Did you use ray.get() to retrieve the values? squares = {squares}'
assert duration < 4.1, f'Did you use Ray to parallelize the work? duration = {duration}' 

AssertionError: Did you use Ray to parallelize the work? duration = 6.007073879241943

## What Is the Optimal Task Granularity

How fine-grained should Ray tasks be? There's no fixed rule of thumb, but Ray clearly adds some overhead for task management and using object stores in a cluster. Therefore, it makes sense that tasks which are too small will perform poorly.

We'll explore this topic over several more lessons, but for now, let's get a sense of the overhead while running in your setup.

We'll continue to use NumPy arrays to create "load", but remove the `sleep` calls:

In [13]:
def noop(n):
    return n

def local_make_array(n):
    return np.random.standard_normal(n)

@ray.remote
def remote_make_array(n):
    return local_make_array(n)

Let's do `trials` runs for each experiment, to average out background noise:

In [14]:
trials=100

First, let's use `noop` to baseline local function calls. Note that we call `print` for the duration, rathern than `pd`, because the overhead is so low the `pd` formatting will print `0.000`:

In [15]:
start = time.time()
[noop(t) for t in range(trials)]
print(f'{time.time() - start} seconds')

0.0002789497375488281 seconds


Let's try the same run with `local_make_array(n)` for `n = 100000`:

In [16]:
start = time.time()
[local_make_array(100000) for _ in range(trials)]
print(f'{time.time() - start} seconds')

0.28948020935058594 seconds


So, we can safely ignore the "noop" overhead for now. For completeness, here's what happens with remote execution:

In [17]:
start = time.time()
ids = [remote_make_array.remote(100000) for _ in range(trials)]
ray.get(ids)
print(f'{time.time() - start} seconds')

0.10944104194641113 seconds


For arrays of 100000, using Ray is faster (at least on this test machine). The benefits of parallel computation, rather than synchronous, already outweight the Ray overhead.

So, let's run some trials with increasingly large array sizes, to compare the performance with local vs. remote execution. First, we'll set up `matplotlib`:

In [18]:
local_durations = []
remote_durations = []
# These n values were determined by experimentation on this test machine. 
# If you are using an old machine, and this cell takes a long time to execute,
# you could set the `trials` value above to a smaller number. 
ns = [i*(10**j) for j in range(2,5) for i in [1,2,3,5,8]]
for n in ns:
    start_local = time.time()
    [local_make_array(n) for _ in range(trials)]
    local_durations.append(time.time() - start_local)
    
    start_remote = time.time()
    ids = [remote_make_array.remote(n) for _ in range(trials)]
    ray.get(ids)
    remote_durations.append(time.time() - start_remote)
(ns, local_durations, remote_durations)

([100,
  200,
  300,
  500,
  800,
  1000,
  2000,
  3000,
  5000,
  8000,
  10000,
  20000,
  30000,
  50000,
  80000],
 [0.0005290508270263672,
  0.0009808540344238281,
  0.0010690689086914062,
  0.001528024673461914,
  0.002393007278442383,
  0.0029311180114746094,
  0.005441188812255859,
  0.00776982307434082,
  0.015281200408935547,
  0.02330303192138672,
  0.029783964157104492,
  0.055117130279541016,
  0.08199191093444824,
  0.14333319664001465,
  0.21774792671203613],
 [0.02898263931274414,
  0.02096080780029297,
  0.01816701889038086,
  0.018893003463745117,
  0.01985907554626465,
  0.019817829132080078,
  0.019781112670898438,
  0.01978588104248047,
  0.01980876922607422,
  0.020447969436645508,
  0.025120973587036133,
  0.03336000442504883,
  0.040186166763305664,
  0.06110262870788574,
  0.0756371021270752])

In [19]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

tooltips = [
    ("name", "$name"),
    ("array size", "$x"),
    ("time", "$y")]
p1 = figure(x_axis_type="log", y_axis_type="log", title="Execution Times", tooltips=tooltips)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'array size'
p1.yaxis.axis_label = 'time'

p1.line(ns, local_durations, color='#A6CEE3', legend_label='local', name='local')
p1.circle(ns, local_durations, color='darkgrey', size=4)
p1.line(ns, remote_durations, color='#B2DF8A', legend_label='remote', name='remote')
p1.square(ns, remote_durations, color='darkgrey', size=4)
p1.legend.location = "top_left"

show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

Here's a static image from a test run, in case the Bokeh plot isn't working:
![Execution Times: Local versus Remote](../images/Execution-Times-Local-v-Remote.png)

Let's confirm what the graph shows as the crossing point:

In [20]:
i=0
while i < len(ns) and local_durations[i] < remote_durations[i]:
    i=i+1
print('The Ray times are faster starting at n = {:d}, local = {:6.3f} vs. remote = {:6.3f}'.format(
    ns[i], local_durations[i], remote_durations[i]))

The Ray times are faster starting at n = 8000, local =  0.023 vs. remote =  0.020


## How Distributed Task Management Works

> **Note:** If you just want to learn the Ray API, you can safely skip the rest of this lesson (notebook) for now. It continues the exploration of how Ray works internally, which we started in the previous lesson. However, you should come back to this material at some point, so you'll develop a better understanding of how Ray works.

At the end of the last lesson, we examined Ray task scheduling at a high-level, by watching the Ray Dashboard and analyzing the performance times. Now we'll walk through some images that show the process Ray follows to place tasks around a cluster. 

Assume we will invoke the `make_array` task twice, then invoke `add_arrays` to sum the returned NumPy arrays. Graphically, it looks as follows:

![Ray under the hood 1](../images/Ray-Cluster/Ray-Cluster.001.jpeg)

How does this get scheduled in a cluster? Here we'll assume a three-node cluster that has resources for running two Ray worker tasks per node (under powered compared to what we learned using Ray Dashboard last lesson!).
![Ray under the hood 2](../images/Ray-Cluster/Ray-Cluster.002.jpeg)

First, assume that the driver program is running on Node1. So it will invoke the local scheduler to schedule the three tasks.
![Ray under the hood 3](../images/Ray-Cluster/Ray-Cluster.003.jpeg)

Immediately the ids for the task futures are returned. The _Global Control Store_ tracks where every task is running and every object is stored in the local _Object Stores_.
![Ray under the hood 4](../images/Ray-Cluster/Ray-Cluster.004.jpeg)

Suppose the local scheduler has available capacity in the first worker on the same node. It schedules the first `make_array` task there.
![Ray under the hood 5](../images/Ray-Cluster/Ray-Cluster.005.jpeg)

It decides to schedule the second `make_array` task in a worker on node 2.
![Ray under the hood 6](../images/Ray-Cluster/Ray-Cluster.006.jpeg)

When the two tasks finish, they place their result objects in their local object stores.
![Ray under the hood 7](../images/Ray-Cluster/Ray-Cluster.007.jpeg)

Now `add_array` can be scheduled, because the two tasks it depends on are done. Let's suppose it gets scheduled in the second worker on Node 1.
![Ray under the hood 8](../images/Ray-Cluster/Ray-Cluster.008.jpeg)

The first object it needs is already on the same node, in the object store, so the `add_arrays` task can _read it directly from shared memory_. No copying is required to the worker's process space.
![Ray under the hood 9](../images/Ray-Cluster/Ray-Cluster.009.jpeg)

However, the second object is on a different node, so Ray copies it to the local object store. 
![Ray under the hood 10](../images/Ray-Cluster/Ray-Cluster.010.jpeg)

Now it can also be read from shared memory.
![Ray under the hood 11](../images/Ray-Cluster/Ray-Cluster.011.jpeg)

When `add_arrays` is finished, it writes its results to the local object store.
![Ray under the hood 12](../images/Ray-Cluster/Ray-Cluster.012.jpeg)

At this point, if the driver calls `ray.get(id3)`, it will return `obj3`.
![Ray under the hood 13](../images/Ray-Cluster/Ray-Cluster.013.jpeg)

Whew! Hopefully you have a better sense of what Ray does under the hood. Scheduling tasks on other nodes and copying objects between object stores is efficient, but incurs unavoidable network overhead.